# Compound Bow


## Imports

In [66]:
from sympy import *
from sympy.physics.mechanics import *
from IPython.display import display

## Symbols

The retract distance for the bow is given by $x$, and the forward speed by $v$. 
Thus, the single generalized coordinate is $x$, and the corresponding generalized speed is $v$. 
Because the positive direction of the two is opposite, we define the kinematic equation $\dot{x} = -v$, or by the convention used in `sympy`, $\dot{x}+v=0$.

In [67]:
x, v = dynamicsymbols('x, v')
kd = [diff(x) + v]

We will define the origin of the model as a point at the grip of the bow. 
The line is initially set back a constant distance $x_0$ from this point, at which point forces are in equilibrium.

In [68]:
x0 = symbols('x_0')

The limbs of the bow will be represented by rigid bodies with a rotational spring attached at fixed positions $(x_l, y_l)$ and $(x_l, -y_l)$ relative to the origin. 
The length of each limb is $l$, and the undeflected angle relative to the horizontal is $\theta_0$. The effective angular stiffness of each limb is $k$. 
The mass and moment of inertia for each limb are $m_l$ and $I_l$, respectively.

In [69]:
xl, yl, l, theta0, ml, Il = symbols('x_l, y_l, l, \\theta_0, m_l, I_l')

The extent of the limb's angular displacement relative to $\theta_0$ is $\theta$. 

In [70]:
theta = dynamicsymbols('\\theta')

The inner and outer radii at the cam are defined as $r_i$ and $r_o$, respectively. 
These are initially defined as constants, though they may alternately be defined as functions of angular displacement for non-circular geometries.

In [71]:
ri, ro = symbols('r_i, r_o')

The angular displacement for the cams relative to their initial angle is $\phi$.

In [72]:
phi = dynamicsymbols('\phi')

## Frames

The inertial reference frames is defined with basis vectors $\hat{\boldsymbol{i}}$, $\hat{\boldsymbol{j}}$, and $\hat{\boldsymbol{k}}$.

In [73]:
latexs = [r'\hat{\boldsymbol{' + ax + '}}' for ax in ('i', 'j', 'k')]
inertial = ReferenceFrame('N', latexs=latexs)

The limb orientations are defined using frames $A$ and $B$.

In [74]:
A = inertial.orientnew('A', 'Axis', [theta0+theta, inertial.z])
B = inertial.orientnew('B', 'Axis', [-theta0-theta, inertial.z])
#display(A.dcm(inertial))

The cam orientations are defined using frames $C$ and $D$.

In [75]:
C = inertial.orientnew('C', 'Axis', [phi, inertial.z])
D = inertial.orientnew('D', 'Axis', [-phi, inertial.z])

## Points

The origin is designated $O$, and has zero velocity in the inertial frame.

In [76]:
origin = Point('O')
origin.set_vel(inertial, Vector(0))

The point representing the position of the retracted line is designated $p$. 

In [77]:
p = origin.locatenew('p', (x+x0)*inertial.x)
p.set_vel(inertial, -v*inertial.x)

The centers of rotation of the limbs are defined as points designated $a$ and $b$.

In [78]:
a = origin.locatenew('a', xl*inertial.x + yl*inertial.y)
b = origin.locatenew('b', xl*inertial.x - yl*inertial.y)

The centers of rotation of the cams are located at points designated $c$ and $d$

In [79]:
c = a.locatenew('c', l*A.x)
d = b.locatenew('d', l*B.x)
display(d.pos_from(origin))

x_l*N.x - y_l*N.y + l*B.x

The points at which the lines contact the inner cam are designated $e$ and $f$.
For circular cams, we can assume the line is tangent to the edge of the circle, in this case the position of the contact point can be determined from a right triangle with the distance between $c$ and $d$ forming the hypotenuse, and the circle radius forming one edge.
The angle for the contact point relative to the local horizontal axis is denoted $\delta$.
For non-circular cams, a separate function is required to determine this contact point.

In [80]:
rho_cd = c.pos_from(d).dot(inertial.y)
#delta = asin(ri/rho_cd)
delta = symbols('\\delta')
e = c.locatenew('e', -ri*cos(delta)*inertial.x - ri*sin(delta)*inertial.y)
f = d.locatenew('f', -ri*cos(delta)*inertial.x + ri*sin(delta)*inertial.y)

In [81]:
#display(rho_cd)
#display(rho_cd.dot(inertial.y))
#display(delta)
display(e.pos_from(origin))

(-r_i*cos(\delta) + x_l)*N.x + (-r_i*sin(\delta) + y_l)*N.y + l*A.x

The points at which the lines contact the outer cams are designated $g$ and $h$.
Similar to the inner cam, if the cam is circular, then a right triangle is formed, in this case with the hypotenuse formed between the contact point and the aft line where the archer holds their finger. 
The angle relative to local horizontal is denoted $\gamma$.

In [82]:
gamma = symbols('\\gamma')
g = c.locatenew('g', ro*cos(gamma)*inertial.x + ro*sin(gamma)*inertial.y)
h = d.locatenew('h', ro*cos(gamma)*inertial.x + ro*sin(gamma)*inertial.y)
display(h.pos_from(origin))

(r_o*cos(\gamma) + x_l)*N.x + (r_o*sin(\gamma) - y_l)*N.y + l*B.x